In [5]:
#胜率计算
def odds_to_prob(odds1, odds2):
    prob1 = 1 / odds1
    prob2 = 1 / odds2
    total_prob = prob1 + prob2
    return prob1 / total_prob, prob2 / total_prob

prob_1301= odds_to_prob(1.09,7.82)
print(prob_1301)

(0.8776655443322109, 0.122334455667789)


接下来需要将比分数据转化为我们的序号数据，现在读取1301文件的数据

In [63]:
import pandas as pd

# 读取Excel文件
file_path = '2023-wimbledon-points正式版本 测试版本.xlsx'
df = pd.read_excel(file_path)
# 删除GameNo中含有13的行
df = df[~df['GameNo'].astype(str).str.contains('13')]

print(df.columns)
# 定义得分分类函数
def classify_score(row):
    score1 = row['P1Score']  # 第一个选手的得分
    score2 = row['P2Score']  # 第二个选手的得分

    if score1 > 4 or score2 > 4:
        return None  # 如果有一个选手的得分大于4，输出空白值

    # 根据规则进行分类
    if score1 == 0 and score2 == 0:
        return 0
    elif score1 == 1 and score2 == 0:
        return 1
    elif score1 == 0 and score2 == 1:
        return 2
    elif score1 == 2 and score2 == 0:
        return 3
    elif score1 == 1 and score2 == 1:
        return 4
    elif score1 == 0 and score2 == 2:
        return 5
    elif score1 == 3 and score2 == 0:
        return 6
    elif score1 == 2 and score2 == 1:
        return 7
    elif score1 == 1 and score2 == 2:
        return 8
    elif score1 == 0 and score2 == 3:
        return 9
    elif score1 == 4 and score2 == 0:
        return 10
    elif score1 == 3 and score2 == 1:
        return 11
    elif score1 == 2 and score2 == 2:
        return 12
    elif score1 == 1 and score2 == 3:
        return 13
    elif score1 == 0 and score2 == 4:
        return 14
    elif score1 == 3 and score2 == 2:
        return 15
    elif score1 == 2 and score2 == 3:
        return 16
    elif score1 == 3 and score2 == 3:
        return 17
    elif score1 == 4 and score2 == 3:
        return 15
    elif score1 == 3 and score2 == 4:
        return 16
    elif score1 == 4 and score2 == 4:
        return 17
    elif score1 == 4 and score2 == 5:
        return 16
    elif score1 == 5 and score2 == 5:
        return 17
    elif score1 == 6 and score2 == 5:
        return 15
    elif score1 == 6 and score2 == 6:
        return 17
    elif score1 == 7 and score2 == 4:
        return 17
    elif score1 == 7 and score2 == 6:
        return 15
    elif score1 == 7 and score2 == 7:
        return 17
    elif score1 == 6 and score2 == 7:
        return 16
    elif score1 == 8 and score2 == 8:
        return 17
    elif score1 == 8 and score2 == 9:
        return 16
    elif score1 == 9 and score2 == 8:
        return 15
    elif score1 == 9 and score2 == 9:
        return 17
    elif score1 == 10 and score2 == 9:
        return 15
    elif score1 == 9 and score2 == 10:
        return 16
    elif score1 == 10 and score2 == 9:
        return 17
    elif score1 == 4 and score2 == 1:
        return 16
    else:
        return 无

# 在DataFrame中应用得分分类函数
df['ScoreCategory'] = df.apply(classify_score, axis=1)

# 将结果输出到新的Excel文件
output_file_path = 'output_example.xlsx'
df.to_excel(output_file_path, index=False)

Index(['match_id', 'ElapsedTime', 'SetNo', 'P1GamesWon', 'P2GamesWon',
       'SetWinner', 'GameNo', 'GameWinner', 'PointNumber', 'PointWinner',
       'PointServer', 'P1Score', 'P2Score', 'Unnamed: 13', '比分序号',
       'P1PointsWon', 'P2PointsWon', 'P1Ace', 'P2Ace', 'P1Winner', 'P2Winner',
       'P1DoubleFault', 'P2DoubleFault', 'P1UnfErr', 'P2UnfErr', 'P1NetPoint',
       'P2NetPoint', 'P1NetPointWon', 'P2NetPointWon', 'P1BreakPoint',
       'P2BreakPoint', 'P1BreakPointWon', 'P2BreakPointWon', 'Speed_MPH',
       'P1BreakPointMissed', 'P2BreakPointMissed', 'ServeNumber',
       'WinnerShotType', 'P1DistanceRun', 'P2DistanceRun', 'RallyCount',
       'ServeWidth', 'ServeDepth', 'ReturnDepth'],
      dtype='object')


现在开始构造矩阵

结合初始的胜率我们开始预测


In [48]:
import pandas as pd
import numpy as np

# 读取Excel文件
file_path = 'output_example.xlsx'  # 请替换成你的文件路径
df = pd.read_excel(file_path)

# 获取最后一列状态的数据
last_column_data = df.iloc[:, -1]

# 定义函数将一维序列分割成多个矩阵
def split_sequences_to_matrices(sequence):
    matrices = []
    current_matrix = []
    
    for state in sequence:
        current_matrix.append(state)
        
        if state == 0:
            if current_matrix:
                matrices.append(current_matrix[:-1])  # Exclude the last 0 from the current matrix
                current_matrix = [0]
    
    # 添加最后一个矩阵（如果有的话）
    if current_matrix:
        matrices.append(current_matrix[:-1])  # Exclude the last 0 from the last matrix
    
    # 如果最后一组数据不是以0结尾，将其添加到矩阵列表中
    if current_matrix not in matrices:
        matrices.append(current_matrix)
    
    return matrices

# 分割序列为多个矩阵
result_matrices = split_sequences_to_matrices(last_column_data.tolist())

# 打印每个矩阵
for idx, matrix in enumerate(result_matrices, start=1):
    print(f"Matrix {idx}:")
    # Convert all elements to integers and join them with commas
    print(f"[{','.join(map(lambda x: str(int(x)), matrix))}]")
    print()

Matrix 1:
[]

Matrix 2:
[0,2,4,8,12,15,17,15,17,15]

Matrix 3:
[0,2,4,7,12,16,17,16]

Matrix 4:
[0,1,3,7,11]

Matrix 5:
[0,2,5,9]

Matrix 6:
[0,1,3,6,11,15,17,15]

Matrix 7:
[0,2,4,8,13]

Matrix 8:
[0,1,3,6,11]

Matrix 9:
[0,2,4,8,12,15]

Matrix 10:
[0,1,4,7,12,15,17,15,17,16,17,15]

Matrix 11:
[0,2,5,9]

Matrix 12:
[0,1,3,7,12,15,17,16]

Matrix 13:
[0,2,5,9]

Matrix 14:
[0,2,4,7,12,16,17,15]

Matrix 15:
[0,2,5,8,12,15,17,15,17,16]

Matrix 16:
[0,2,4,8,12,15,17,15]

Matrix 17:
[0,1,4,8,13,16,17,15,17,16,17,15]

Matrix 18:
[0,1,3,6]

Matrix 19:
[0,1,4,8,13]

Matrix 20:
[0,1,4,7,11,15]

Matrix 21:
[0,2,5,8,13]

Matrix 22:
[0,1,3,6]

Matrix 23:
[0,1,4,7,12,15]

Matrix 24:
[0,1,4,8,12,15,17,15,17,16,17,16]

Matrix 25:
[0,1,3,7,12,15]

Matrix 26:
[0,1,3,7,11]

Matrix 27:
[0,1,3,6,11]

Matrix 28:
[0,1,4,8,12,16]

Matrix 29:
[0,1,3,6]

Matrix 30:
[0,2,4,8,12,15,17,16]

Matrix 31:
[0,1,3,6,11]

Matrix 32:
[0,1,4,8,13]

Matrix 33:
[0,1,4,8,12,16]

Matrix 34:
[0,2,4,8,13,16,17,15,17,16,17,16,17,

In [49]:
import pandas as pd
import numpy as np

# 读取Excel文件
file_path = 'output_example.xlsx'  # 请替换成你的文件路径
df = pd.read_excel(file_path)

# 获取最后一列状态的数据
last_column_data = df.iloc[:, -1]

# 定义函数将一维序列分割成多个矩阵
def split_sequences_to_matrices(sequence):
    matrices = []
    current_matrix = []
    
    for state in sequence:
        current_matrix.append(state)
        
        if state == 0:
            if current_matrix:
                matrices.append(current_matrix[:-1])  # Exclude the last 0 from the current matrix
                current_matrix = [0]
    
    # 添加最后一个矩阵（如果有的话）
    if current_matrix:
        matrices.append(current_matrix[:-1])  # Exclude the last 0 from the last matrix
    
    return matrices

# 分割序列为多个矩阵
result_matrices = split_sequences_to_matrices(last_column_data.tolist())

# 进一步处理生成的矩阵（除了最后一个矩阵）
for idx, matrix in enumerate(result_matrices[:-1]):
    if len(matrix) > 0:
        last_element = matrix[-1]
        
        if last_element in [6, 11, 15]:
            matrix.append(10)
        elif last_element in [16, 13, 9]:
            matrix.append(14)

# 单独处理最后一个矩阵
last_matrix = result_matrices[-1]
if len(last_matrix) > 0:
    last_element = last_matrix[-1]

    # 进一步处理最后一个矩阵
    if last_element in [6, 11, 15]:
        last_matrix.append(10)
    elif last_element in [16, 13, 9]:
        last_matrix.append(14)

# 打印处理后的每个矩阵
for idx, matrix in enumerate(result_matrices, start=1):
    # Check if the matrix has at least one number
    if any(isinstance(elem, (int, float)) for elem in matrix):
        print(f"Matrix {idx}:")
        # Convert all elements to integers and join them with commas
        print(f"[{','.join(map(lambda x: str(int(x)), matrix))}]")
        print()

Matrix 2:
[0,2,4,8,12,15,17,15,17,15,10]

Matrix 3:
[0,2,4,7,12,16,17,16,14]

Matrix 4:
[0,1,3,7,11,10]

Matrix 5:
[0,2,5,9,14]

Matrix 6:
[0,1,3,6,11,15,17,15,10]

Matrix 7:
[0,2,4,8,13,14]

Matrix 8:
[0,1,3,6,11,10]

Matrix 9:
[0,2,4,8,12,15,10]

Matrix 10:
[0,1,4,7,12,15,17,15,17,16,17,15,10]

Matrix 11:
[0,2,5,9,14]

Matrix 12:
[0,1,3,7,12,15,17,16,14]

Matrix 13:
[0,2,5,9,14]

Matrix 14:
[0,2,4,7,12,16,17,15,10]

Matrix 15:
[0,2,5,8,12,15,17,15,17,16,14]

Matrix 16:
[0,2,4,8,12,15,17,15,10]

Matrix 17:
[0,1,4,8,13,16,17,15,17,16,17,15,10]

Matrix 18:
[0,1,3,6,10]

Matrix 19:
[0,1,4,8,13,14]

Matrix 20:
[0,1,4,7,11,15,10]

Matrix 21:
[0,2,5,8,13,14]

Matrix 22:
[0,1,3,6,10]

Matrix 23:
[0,1,4,7,12,15,10]

Matrix 24:
[0,1,4,8,12,15,17,15,17,16,17,16,14]

Matrix 25:
[0,1,3,7,12,15,10]

Matrix 26:
[0,1,3,7,11,10]

Matrix 27:
[0,1,3,6,11,10]

Matrix 28:
[0,1,4,8,12,16,14]

Matrix 29:
[0,1,3,6,10]

Matrix 30:
[0,2,4,8,12,15,17,16,14]

Matrix 31:
[0,1,3,6,11,10]

Matrix 32:
[0,1,4,8,13,1

In [55]:
import pandas as pd
import numpy as np

# 读取Excel文件
file_path = 'output_example.xlsx'  # 请替换成你的文件路径
df = pd.read_excel(file_path)

# 获取最后一列状态的数据
last_column_data = df.iloc[:, -1]

# 定义函数将一维序列分割成多个矩阵
def split_sequences_to_matrices(sequence):
    matrices = []
    current_matrix = []
    
    for state in sequence:
        current_matrix.append(state)
        
        if state == 0:
            if current_matrix:
                matrices.append(current_matrix[:-1])  # Exclude the last 0 from the current matrix
                current_matrix = [0]
    
    # 添加最后一个矩阵（如果有的话）
    if current_matrix:
        matrices.append(current_matrix)  # Include the last 0 in the last matrix
    
    return matrices

# 分割序列为多个矩阵
result_matrices = split_sequences_to_matrices(last_column_data.tolist())

# 手动处理最后一个矩阵
last_matrix = result_matrices[-1]
if len(last_matrix) > 0:
    last_element = last_matrix[-1]


# 进一步处理生成的矩阵（除了最后一个矩阵）
for idx, matrix in enumerate(result_matrices[:-1]):
    if len(matrix) > 0:
        last_element = matrix[-1]
        
        if last_element in [6, 11, 15]:
            matrix.append(10)
        elif last_element in [16, 13, 9]:
            matrix.append(14)

# 手动处理最后一个矩阵
last_matrix = result_matrices[-1]
if len(last_matrix) > 0:
    last_element = last_matrix[-1]
    
    if last_element in [6, 11, 15]:
        last_matrix.append(10)
    elif last_element in [16, 13, 9]:
        last_matrix.append(14)

# 打印处理后的每个矩阵
for idx, matrix in enumerate(result_matrices, start=1):
    # Check if the matrix has at least one number
    if any(isinstance(elem, (int, float)) for elem in matrix):
        print(f"Matrix {idx}:")
        # Convert all elements to integers and join them with commas
        print(f"[{','.join(map(lambda x: str(int(x)), matrix))}]")
        print()

Matrix 2:
[0,2,4,8,12,15,17,15,17,15,10]

Matrix 3:
[0,2,4,7,12,16,17,16,14]

Matrix 4:
[0,1,3,7,11,10]

Matrix 5:
[0,2,5,9,14]

Matrix 6:
[0,1,3,6,11,15,17,15,10]

Matrix 7:
[0,2,4,8,13,14]

Matrix 8:
[0,1,3,6,11,10]

Matrix 9:
[0,2,4,8,12,15,10]

Matrix 10:
[0,1,4,7,12,15,17,15,17,16,17,15,10]

Matrix 11:
[0,2,5,9,14]

Matrix 12:
[0,1,3,7,12,15,17,16,14]

Matrix 13:
[0,2,5,9,14]

Matrix 14:
[0,2,4,7,12,16,17,15,10]

Matrix 15:
[0,2,5,8,12,15,17,15,17,16,14]

Matrix 16:
[0,2,4,8,12,15,17,15,10]

Matrix 17:
[0,1,4,8,13,16,17,15,17,16,17,15,10]

Matrix 18:
[0,1,3,6,10]

Matrix 19:
[0,1,4,8,13,14]

Matrix 20:
[0,1,4,7,11,15,10]

Matrix 21:
[0,2,5,8,13,14]

Matrix 22:
[0,1,3,6,10]

Matrix 23:
[0,1,4,7,12,15,10]

Matrix 24:
[0,1,4,8,12,15,17,15,17,16,17,16,14]

Matrix 25:
[0,1,3,7,12,15,10]

Matrix 26:
[0,1,3,7,11,10]

Matrix 27:
[0,1,3,6,11,10]

Matrix 28:
[0,1,4,8,12,16,14]

Matrix 29:
[0,1,3,6,10]

Matrix 30:
[0,2,4,8,12,15,17,16,14]

Matrix 31:
[0,1,3,6,11,10]

Matrix 32:
[0,1,4,8,13,1

In [64]:
# 打印处理后的每个矩阵及其类型
for idx, matrix in enumerate(observed_sequences, start=1):
    # Check if the matrix has at least one number
    if any(isinstance(elem, str) for elem in matrix):
        print(f"Matrix {idx}:")
        # Print the type of the matrix
        print(f"Type: {type(matrix)}")
        # Join all elements with commas and print as a string
        print(f"[{','.join(matrix)}]")
        print()

Matrix 2:
Type: <class 'list'>
[0,2,4,8,12,15,17,15,17,15,10]

Matrix 3:
Type: <class 'list'>
[0,2,4,7,12,16,17,16,14]

Matrix 4:
Type: <class 'list'>
[0,1,3,7,11,10]

Matrix 5:
Type: <class 'list'>
[0,2,5,9,14]

Matrix 6:
Type: <class 'list'>
[0,1,3,6,11,15,17,15,10]

Matrix 7:
Type: <class 'list'>
[0,2,4,8,13,14]

Matrix 8:
Type: <class 'list'>
[0,1,3,6,11,10]

Matrix 9:
Type: <class 'list'>
[0,2,4,8,12,15,10]

Matrix 10:
Type: <class 'list'>
[0,1,4,7,12,15,17,15,17,16,17,15,10]

Matrix 11:
Type: <class 'list'>
[0,2,5,9,14]

Matrix 12:
Type: <class 'list'>
[0,1,3,7,12,15,17,16,14]

Matrix 13:
Type: <class 'list'>
[0,2,5,9,14]

Matrix 14:
Type: <class 'list'>
[0,2,4,7,12,16,17,15,10]

Matrix 15:
Type: <class 'list'>
[0,2,5,8,12,15,17,15,17,16,14]

Matrix 16:
Type: <class 'list'>
[0,2,4,8,12,15,17,15,10]

Matrix 17:
Type: <class 'list'>
[0,1,4,8,13,16,17,15,17,16,17,15,10]

Matrix 18:
Type: <class 'list'>
[0,1,3,6,10]

Matrix 19:
Type: <class 'list'>
[0,1,4,8,13,14]

Matrix 20:
Type: <

初始带完已经实现，这个实现的是除了Game13以外的数据，我们现在把以6,11,15为结尾的后面加一个10，以16,13,9为结尾的矩阵最后加一个14

# 将所有矩阵内的数值转换成带单引号的字符串形式
observed_sequences = [["'" + str(num) + "'" for num in row] for row in observed_sequences]

# 打印处理后的每个矩阵及其类型
for idx, matrix in enumerate(observed_sequences, start=1):
    # Check if the matrix has at least one number
    if any(isinstance(elem, str) for elem in matrix):
        print(f"Matrix {idx}:")
        # Print the type of the matrix
        print(f"Type: {type(matrix)}")
        # Join all elements with commas and print as a string
        print(f"[{','.join(matrix)}]")
        print()

In [66]:
# 将所有矩阵内的数值转换成带单引号的字符串形式
observed_sequences = [["'" + str(num) + "'" for num in row] for row in observed_sequences]

# 打印处理后的每个矩阵及其类型
for idx, matrix in enumerate(observed_sequences, start=1):
    # Check if the matrix has at least one number
    if any(isinstance(elem, str) for elem in matrix):
        print(f"Matrix {idx}:")
        # Print the type of the matrix
        print(f"Type: {type(matrix)}")
        # Join all elements with commas and print as a string
        print(f"[{','.join(matrix)}]")
        print()

Matrix 2:
Type: <class 'list'>
['0','2','4','8','12','15','17','15','17','15','10']

Matrix 3:
Type: <class 'list'>
['0','2','4','7','12','16','17','16','14']

Matrix 4:
Type: <class 'list'>
['0','1','3','7','11','10']

Matrix 5:
Type: <class 'list'>
['0','2','5','9','14']

Matrix 6:
Type: <class 'list'>
['0','1','3','6','11','15','17','15','10']

Matrix 7:
Type: <class 'list'>
['0','2','4','8','13','14']

Matrix 8:
Type: <class 'list'>
['0','1','3','6','11','10']

Matrix 9:
Type: <class 'list'>
['0','2','4','8','12','15','10']

Matrix 10:
Type: <class 'list'>
['0','1','4','7','12','15','17','15','17','16','17','15','10']

Matrix 11:
Type: <class 'list'>
['0','2','5','9','14']

Matrix 12:
Type: <class 'list'>
['0','1','3','7','12','15','17','16','14']

Matrix 13:
Type: <class 'list'>
['0','2','5','9','14']

Matrix 14:
Type: <class 'list'>
['0','2','4','7','12','16','17','15','10']

Matrix 15:
Type: <class 'list'>
['0','2','5','8','12','15','17','15','17','16','14']

Matrix 16:
Type: <c

In [70]:
# 将所有矩阵放到一个新的列表中
combined_list = [matrix for matrix in observed_sequences]

# 打印新的列表及其类型
print("Combined List:")
print(f"Type: {type(combined_list)}")
print(combined_list)

Combined List:
Type: <class 'list'>
[[], ["'''0'''", "'''2'''", "'''4'''", "'''8'''", "'''12'''", "'''15'''", "'''17'''", "'''15'''", "'''17'''", "'''15'''", "'''10'''"], ["'''0'''", "'''2'''", "'''4'''", "'''7'''", "'''12'''", "'''16'''", "'''17'''", "'''16'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''7'''", "'''11'''", "'''10'''"], ["'''0'''", "'''2'''", "'''5'''", "'''9'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''6'''", "'''11'''", "'''15'''", "'''17'''", "'''15'''", "'''10'''"], ["'''0'''", "'''2'''", "'''4'''", "'''8'''", "'''13'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''6'''", "'''11'''", "'''10'''"], ["'''0'''", "'''2'''", "'''4'''", "'''8'''", "'''12'''", "'''15'''", "'''10'''"], ["'''0'''", "'''1'''", "'''4'''", "'''7'''", "'''12'''", "'''15'''", "'''17'''", "'''15'''", "'''17'''", "'''16'''", "'''17'''", "'''15'''", "'''10'''"], ["'''0'''", "'''2'''", "'''5'''", "'''9'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''7'''", "'''12'''"

In [71]:
# 将所有矩阵放到一个新的列表中，不包含引号
combined_list = [matrix for matrix in observed_sequences]

# 打印新的列表及其类型
print("Combined List:")
print(f"Type: {type(combined_list)}")
print(combined_list)

Combined List:
Type: <class 'list'>
[[], ["'''0'''", "'''2'''", "'''4'''", "'''8'''", "'''12'''", "'''15'''", "'''17'''", "'''15'''", "'''17'''", "'''15'''", "'''10'''"], ["'''0'''", "'''2'''", "'''4'''", "'''7'''", "'''12'''", "'''16'''", "'''17'''", "'''16'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''7'''", "'''11'''", "'''10'''"], ["'''0'''", "'''2'''", "'''5'''", "'''9'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''6'''", "'''11'''", "'''15'''", "'''17'''", "'''15'''", "'''10'''"], ["'''0'''", "'''2'''", "'''4'''", "'''8'''", "'''13'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''6'''", "'''11'''", "'''10'''"], ["'''0'''", "'''2'''", "'''4'''", "'''8'''", "'''12'''", "'''15'''", "'''10'''"], ["'''0'''", "'''1'''", "'''4'''", "'''7'''", "'''12'''", "'''15'''", "'''17'''", "'''15'''", "'''17'''", "'''16'''", "'''17'''", "'''15'''", "'''10'''"], ["'''0'''", "'''2'''", "'''5'''", "'''9'''", "'''14'''"], ["'''0'''", "'''1'''", "'''3'''", "'''7'''", "'''12'''"

In [73]:
# 使用 eval() 将字符串形式的数字转换为正常的数字
combined_list = [[eval(num) for num in matrix] for matrix in observed_sequences]

# 打印新的列表及其类型
print("Combined List:")
print(f"Type: {type(combined_list)}")
print(combined_list)

Combined List:
Type: <class 'list'>
[[], ['0', '2', '4', '8', '12', '15', '17', '15', '17', '15', '10'], ['0', '2', '4', '7', '12', '16', '17', '16', '14'], ['0', '1', '3', '7', '11', '10'], ['0', '2', '5', '9', '14'], ['0', '1', '3', '6', '11', '15', '17', '15', '10'], ['0', '2', '4', '8', '13', '14'], ['0', '1', '3', '6', '11', '10'], ['0', '2', '4', '8', '12', '15', '10'], ['0', '1', '4', '7', '12', '15', '17', '15', '17', '16', '17', '15', '10'], ['0', '2', '5', '9', '14'], ['0', '1', '3', '7', '12', '15', '17', '16', '14'], ['0', '2', '5', '9', '14'], ['0', '2', '4', '7', '12', '16', '17', '15', '10'], ['0', '2', '5', '8', '12', '15', '17', '15', '17', '16', '14'], ['0', '2', '4', '8', '12', '15', '17', '15', '10'], ['0', '1', '4', '8', '13', '16', '17', '15', '17', '16', '17', '15', '10'], ['0', '1', '3', '6', '10'], ['0', '1', '4', '8', '13', '14'], ['0', '1', '4', '7', '11', '15', '10'], ['0', '2', '5', '8', '13', '14'], ['0', '1', '3', '6', '10'], ['0', '1', '4', '7', '12', '1

去除空白list


In [74]:
# 去除空白列表
combined_list = [[eval(num) for num in matrix] for matrix in observed_sequences if matrix]

# 打印新的列表及其类型
print("Combined List:")
print(f"Type: {type(combined_list)}")
print(combined_list)

Combined List:
Type: <class 'list'>
[['0', '2', '4', '8', '12', '15', '17', '15', '17', '15', '10'], ['0', '2', '4', '7', '12', '16', '17', '16', '14'], ['0', '1', '3', '7', '11', '10'], ['0', '2', '5', '9', '14'], ['0', '1', '3', '6', '11', '15', '17', '15', '10'], ['0', '2', '4', '8', '13', '14'], ['0', '1', '3', '6', '11', '10'], ['0', '2', '4', '8', '12', '15', '10'], ['0', '1', '4', '7', '12', '15', '17', '15', '17', '16', '17', '15', '10'], ['0', '2', '5', '9', '14'], ['0', '1', '3', '7', '12', '15', '17', '16', '14'], ['0', '2', '5', '9', '14'], ['0', '2', '4', '7', '12', '16', '17', '15', '10'], ['0', '2', '5', '8', '12', '15', '17', '15', '17', '16', '14'], ['0', '2', '4', '8', '12', '15', '17', '15', '10'], ['0', '1', '4', '8', '13', '16', '17', '15', '17', '16', '17', '15', '10'], ['0', '1', '3', '6', '10'], ['0', '1', '4', '8', '13', '14'], ['0', '1', '4', '7', '11', '15', '10'], ['0', '2', '5', '8', '13', '14'], ['0', '1', '3', '6', '10'], ['0', '1', '4', '7', '12', '15', 

In [77]:
import numpy as np

def calculate_transition_matrix_from_multiple_sequences(sequences, states):
    num_states = len(states)
    transition_matrix = np.zeros((num_states, num_states))

    for sequence in sequences:
        for i in range(len(sequence) - 1):
            current_state = sequence[i]
            next_state = sequence[i + 1]

            current_index = states.index(current_state)
            next_index = states.index(next_state)

            transition_matrix[current_index, next_index] += 1

    # Normalize the transition matrix to obtain probabilities
    row_sums = transition_matrix.sum(axis=1, keepdims=True)
    transition_matrix = transition_matrix / row_sums

    return transition_matrix

# Example usage:
states = ['0', '1', '2', '3', '4', '5','6','7','8','9','10','11','12','13','14','15','16','17']
observed_sequences = combined_list
transition_matrix = np.nan_to_num(calculate_transition_matrix_from_multiple_sequences(observed_sequences, states))

print("Transition Matrix:")
print(transition_matrix)



# 预测某一状态到另一状态的概率值
def dfs(matrix, start, end, visited, path, all_paths):
    if start == end:
        all_paths.append(list(path))
        return
    visited[start] = True
    for i, prob in enumerate(matrix[start]):
        if prob > 0 and not visited[i]:
            path.append(prob)
            dfs(matrix, i, end, visited, path, all_paths)
            path.pop()
    visited[start] = False

def average_path_probability(matrix, start, end):
    visited = [False] * len(matrix)
    all_paths = []
    dfs(matrix, start, end, visited, [], all_paths)
    if not all_paths:
        return 0  # 如果没有路径，返回0
    total_prob = 0
    for path in all_paths:
        path_prob = 1
        for prob in path:
            path_prob *= prob
        total_prob += path_prob
    return total_prob / len(all_paths)

# Example usage:
start_state = 0
end_state = 10

print(average_path_probability(transition_matrix, start_state, end_state))


Transition Matrix:
[[0.         0.53500432 0.46499568 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.56704362 0.43295638 0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.44981413 0.55018587
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.62678063 0.37321937 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.         0.52352941 0.47647059 0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.  

/var/folders/bt/qcg_frss46xfvvsc6gdw2gd80000gn/T/ipykernel_10770/2834402821.py:19: RuntimeWarning: invalid value encountered in divide
  transition_matrix = transition_matrix / row_sums


In [78]:
import pandas as pd
import numpy as np

# ...（之前的代码不变）...

# 将转移矩阵转换为 DataFrame
transition_matrix_df = pd.DataFrame(transition_matrix, index=states, columns=states)

# 将 DataFrame 写入 Excel 文件
excel_output_path = 'transition_matrix_output.xlsx'
transition_matrix_df.to_excel(excel_output_path, index=True)

# 打印输出 Excel 文件路径
print(f"Transition Matrix Excel file has been saved to: {excel_output_path}")

# ...（之后的代码不变）...

Transition Matrix Excel file has been saved to: transition_matrix_output.xlsx


In [80]:
import numpy as np

def calculate_transition_matrix_from_multiple_sequences(sequences, states):
    num_states = len(states)
    transition_matrix = np.zeros((num_states, num_states))

    for sequence in sequences:
        for i in range(len(sequence) - 1):
            current_state = sequence[i]
            next_state = sequence[i + 1]

            current_index = states.index(current_state)
            next_index = states.index(next_state)

            transition_matrix[current_index, next_index] += 1

    # Normalize the transition matrix to obtain probabilities
    row_sums = transition_matrix.sum(axis=1, keepdims=True)
    transition_matrix = transition_matrix / row_sums

    return transition_matrix

def dfs(matrix, start, end, visited, path, all_paths):
    if start == end:
        all_paths.append(list(path))
        return
    visited[start] = True
    for i, prob in enumerate(matrix[start]):
        if prob > 0 and not visited[i]:
            path.append(prob)
            dfs(matrix, i, end, visited, path, all_paths)
            path.pop()
    visited[start] = False

def average_path_probability(matrix, start, end):
    visited = [False] * len(matrix)
    all_paths = []
    dfs(matrix, start, end, visited, [], all_paths)
    if not all_paths:
        return 0  # 如果没有路径，返回0
    total_prob = 0
    for path in all_paths:
        path_prob = 1
        for prob in path:
            path_prob *= prob
        total_prob += path_prob
    return total_prob / len(all_paths)

# Example usage:
states = ['0', '1', '2', '3', '4', '5','6','7','8','9','10','11','12','13','14','15','16','17']
observed_sequences = combined_list
transition_matrix = np.nan_to_num(calculate_transition_matrix_from_multiple_sequences(observed_sequences, states))

# 预测每一个状态到所有状态的概率并输出成一个matrix
num_states = len(states)
prediction_matrix = np.zeros((num_states, num_states))

for start_state in range(num_states):
    for end_state in range(num_states):
        prediction_matrix[start_state, end_state] = average_path_probability(transition_matrix, start_state, end_state)

print("Prediction Matrix:")
print(prediction_matrix)

Prediction Matrix:
[[1.         0.53500432 0.46499568 0.30337079 0.22039758 0.25583405
  0.19014693 0.11466436 0.1016998  0.16076059 0.01783621 0.06179775
  0.04868914 0.05509939 0.01618098 0.01442645 0.01408812 0.01035827]
 [0.         1.         0.         0.56704362 0.43295638 0.
  0.35541195 0.21914853 0.20629098 0.         0.04572491 0.11732797
  0.09754214 0.11629435 0.01915019 0.03179392 0.02503969 0.02064775]
 [0.         0.         1.         0.         0.44981413 0.55018587
  0.         0.23549092 0.20939208 0.34572491 0.02021972 0.12662075
  0.09718974 0.11339145 0.04482757 0.02546909 0.03178501 0.0207957 ]
 [0.         0.         0.         1.         0.         0.
  0.62678063 0.37321937 0.         0.         0.11730631 0.20290197
  0.17254363 0.         0.01903059 0.06340915 0.03426936 0.03549322]
 [0.         0.         0.         0.         1.         0.
  0.         0.52352941 0.47647059 0.         0.05758553 0.28149571
  0.22494953 0.26860523 0.05526372 0.06702605 0.0

/var/folders/bt/qcg_frss46xfvvsc6gdw2gd80000gn/T/ipykernel_10770/1438140122.py:19: RuntimeWarning: invalid value encountered in divide
  transition_matrix = transition_matrix / row_sums


输出为Excel


In [83]:
import numpy as np
import pandas as pd

# Your existing code...

# Example usage:
states = ['0', '1', '2', '3', '4', '5','6','7','8','9','10','11','12','13','14','15','16','17']
observed_sequences = combined_list
transition_matrix = np.nan_to_num(calculate_transition_matrix_from_multiple_sequences(observed_sequences, states))

# Predict each state's probability to all states and output as a matrix
num_states = len(states)
prediction_matrix = np.zeros((num_states, num_states))

for start_state in range(num_states):
    for end_state in range(num_states):
        prediction_matrix[start_state, end_state] = average_path_probability(transition_matrix, start_state, end_state)

# Convert the matrix to a DataFrame
prediction_df = pd.DataFrame(prediction_matrix, index=states, columns=states)

# Save the DataFrame to an Excel file
output_excel_path = 'prediction_matrix.xlsx'
prediction_df.to_excel(output_excel_path)

print("Prediction Matrix saved to:", output_excel_path)

Prediction Matrix saved to: prediction_matrix.xlsx


/var/folders/bt/qcg_frss46xfvvsc6gdw2gd80000gn/T/ipykernel_10770/1438140122.py:19: RuntimeWarning: invalid value encountered in divide
  transition_matrix = transition_matrix / row_sums
